In [ ]:
from functions import *
from parameters import *

from sklearn.linear_model import Lasso
from sklearn.linear_model import lasso_path
from tqdm import tqdm

The raw data, as a CSV file.

In [ ]:
LOG( "Data (data-frame)" )
filename = "raw/data_ml.csv"
LOG( f"Reading {filename} [20 seconds]" )
d = pd.read_csv(filename)
d['date'] = pd.to_datetime( d['date'] )

predictors = list( signs.keys() )
target = 'R1M_Usd'

Rather than a data-frame, it is often easier to have a list of matrices, one per variable, 
with one row per stock and one column per date: we can easily combine them or apply some function to each row, or each column.

In [ ]:
LOG( "Data (list of matrices)" )
LOG( "Read data/data_ml.pickle" )
dd = load( "data/data_ml.pickle" )

# Single signals
For any of the input variables, we can divide the universe into quintiles, long the top quintile, and short the bottom quintile.

In [ ]:
LOG( "Backtest a single signal" )
trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

signal = predictors[0]
LOG( f"  {signal}" )
x = dd[signal].copy() * signs[signal]
x.fillna(.5, inplace=True)  ## Replace missing values with the median (0.5, since the predictors are uniform)
x = np.where( dd['universe'], 1, np.nan ) * x   # Only invest in stocks in the universe
r = signal_backtest(x, y, date = DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( r['dates'], r['prices'].iloc[i,:], color = quintile_colours[i] )
ax.set_yscale('log')
ax.set_title(signal if signs[signal]>=0 else f"- {signal}")
plt.show()

r['performance']

# Lasso
We try to forecast the 1-month forward return from all the input variables. 

As we let the scale of the $L^1$ penalty vary, we have a family of increasingly complex models, from the intercept-only one, to the (unpenalized) least-squares model.

In [ ]:
LOG( "Training data" )
i = np.array([ str(u) < DATE1 for u in d['date'] ]) 
train = d[i].copy()
x = train[ predictors ]
y = np.log1p(train[ target ])

LOG( "Clean the data" )
i = np.isfinite(y)
x = x[i]
y = y[i]

x = x.fillna(.5)

LOG( "Lasso" )
alphas, coef, _ = lasso_path( X = x, y = y, max_iter = 10_000 )

In [ ]:
trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

LOG( "Loop: backtests of the lasso models [10 minutes]" )
res = {}
for j in tqdm(range(len(alphas))):
    signal = np.zeros( shape = dd[ list(dd.keys())[0] ].shape )
    for i,predictor in enumerate(predictors):
        signal += coef[i,j] * dd[predictor].fillna(.5)
    signal = np.where( dd['universe'], 1, np.nan ) * signal
    r = signal_backtest(signal, y, date=DATE1)
    r['performance'  ]['alpha'] = alphas[j]
    r['in-sample'    ]['alpha'] = alphas[j]
    r['out-of-sample']['alpha'] = alphas[j]
    r['performance'  ]['period'] = 'all'
    r['in-sample'    ]['period'] = 'in-sample'
    r['out-of-sample']['period'] = 'out-of-sample'    
    res[ f"{alphas[j]} all" ] = r['performance']
    res[ f"{alphas[j]} in"  ] = r['in-sample']
    res[ f"{alphas[j]} out" ] = r['out-of-sample']
    
    if j == 20:
        # Wealth curves
        fig, ax = plt.subplots()
        for i in range(6):
            ax.plot( r['dates'], r['prices'].iloc[i,:], color = quintile_colours[i] )
        ax.set_yscale('log')
        ax.set_title( f"alpha={alphas[j]:5.3}")
        plt.show()
        
res = pd.concat( res.values() )

We expect the in-sample performance to increase, and the out-of-sample performance to increase and then decrease -- 
this is not what we see here: the out-of-sample performance does not decrease, and a linear model performs extremely well.

In [ ]:
i1 = ( res['portfolio'] == 'LS' ) & ( res['period'] == 'in-sample' )
i2 = ( res['portfolio'] == 'LS' ) & ( res['period'] == 'out-of-sample' )
i3 = ( res['portfolio'] == 'LS' ) & ( res['period'] == 'all' )
fig, axs = plt.subplots(1,3, figsize=(15,5))
for j,key in enumerate(['Information Ratio', 'CAGR', 'Annualized Volatility']):
    ax = axs[j]
    ax.plot( res[i1]['alpha'], res[i1][key], label = "in-sample" )
    ax.plot( res[i2]['alpha'], res[i2][key], label = "out-of-sample" )
    #ax.plot( res[i3]['alpha'], res[i3][key], label = "all" )
    ax.set_xlabel('Alpha')
    ax.set_xscale('log')
    ax.invert_xaxis()
    ax.set_title(key)
    ax.legend()
plt.show()

number_of_predictors = ( coef != 0 ).sum(axis=0)
fig, ax = plt.subplots()
ax.plot( alphas, number_of_predictors )
ax.set_xlabel('Alpha')
ax.set_xscale('log')
ax.invert_xaxis()
ax.set_title('Number of predictors')
plt.show()

# Constrained regression

To improve the performance of the linear models, to prevent it from overfitting the data, 
and to ensure the model remains interpretable, we can add constraints on the signs of the coefficients of the regression.
Indeed we know in advance whether each predictor has a positive or negative impact on future returns:
for instance, volatility has a negative impact, while earnings yield has a positive impact.

In [ ]:
from scipy.optimize import lsq_linear

LOG( "Training data" )
i = np.array([ str(u) < DATE1 for u in d['date'] ]) 
train = d[i].copy()
x = train[ predictors ]
y = np.log1p(train[ target ])

LOG( "Clean the data" )
i = np.isfinite(y)
x = x[i]
y = y[i]

x = x.fillna(.5)

LOG( "Fit the model" )
r = lsq_linear(
    x, y,
    (
        [ 0      if signs[u] > 0 else -np.inf for u in predictors ],
        [ np.inf if signs[u] > 0 else 0       for u in predictors ],
    )
)

# It is not supposed to be that sparse: usually, 2/3 of the coefficients are non-zero...
w = np.round( 1e6 * r.x )
{ p: w[i] for i,p in enumerate(predictors) if w[i] != 0 }

In [ ]:
LOG( "Data for the backtest" )
trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

signal = np.zeros( shape = dd[ list(dd.keys())[0] ].shape )
for i,predictor in enumerate(predictors):
    signal += r.x[i] * dd[predictor].fillna(.5)
    #signal += r_rev.x[i] * dd[predictor].fillna(.5)
    #signal += r_ols.x[i] * dd[predictor].fillna(.5)
signal = np.where( dd['universe'], 1, np.nan ) * signal
res = signal_backtest(signal, y, date=DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.set_yscale('log')
ax.set_title('Constrained regression')
plt.show()

res['out-of-sample']

# Deep learning: linear model
This is equivalent to the models above, but implemented with a neural network.
* Linear model
* Add mini-batches
* Add sign constraints

In [ ]:
# TODO

# Deep learning: lattice networks

In [ ]:
# TODO

# Deep learning: optimization

In [ ]:
# TODO